In [4]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached llama_index_llms_llama_cpp-0.1.4-py3-none-any.whl.metadata (695 bytes)
Using cached llama_index_llms_llama_cpp-0.1.4-py3-none-any.whl (6.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/home/aswin/Desktop/GIT/llama_rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install llama-cpp-python

In [3]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-7B-chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf"

llm = LlamaCPP(
   
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
 
    context_window=3900,

    generate_kwargs={},

    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /tmp/llama_index/models/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_cou

In [7]:
!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [4]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "2310"
port = "5432"
user = "postgres"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host="localhost",
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")
    c.execute(f'GRANT CONNECT ON DATABASE {db_name} TO {user}')
    

In [5]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host="localhost",
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

In [58]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

mkdir: cannot create directory ‘data’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-07-20 17:41:30--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.131.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2307.09288 [following]
--2024-07-20 17:41:31--  http://arxiv.org/pdf/2307.09288
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  2.15MB/s    in 13s     

2024-07-20 17:41:45 (993 KB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



In [6]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [7]:
loader = PyMuPDFReader()
documents = loader.load(file_path="/home/aswin/Desktop/GIT/llama_rag/Research_Regulations-2023-1166-2023-KTU.pdf")

In [8]:
from llama_index.core.node_parser import SentenceSplitter

In [9]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [10]:
text_chunks = []

doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [11]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [12]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [13]:
vector_store.add(nodes)

['b5c19452-cce6-44eb-b051-7af5c451a905',
 '9f747596-3f9e-41b1-a73f-79ef8cd02492',
 'efe12daf-27db-4e7d-90d1-02754112cf22',
 'acc2c021-f6cc-4226-95ba-88895870722a',
 '978a9a52-c74d-43de-9a21-7ee629b847f4',
 '4c4f930c-60ef-43c9-9032-54c6a68cc092',
 '7fd9eadb-c764-4fa2-96b7-973a72b83a61',
 'fa63ee0a-e82f-4ff3-be83-cb187df751eb',
 '6e99ea8d-c9ca-4432-83be-2413caf21e65',
 'cdacdf96-9602-41ba-85a2-2ed9fb032729',
 'b53cea05-397d-4cb4-9b02-602a8c0d181e',
 '4bed9870-047e-48f6-89d6-54b5ac62587f',
 'd8b97f5e-fca1-42ee-ae35-75cfa8fb6260',
 '8361e18d-caf9-4dbf-a35c-87c52ab8552a',
 '0eab03c5-0683-49a3-91c2-1246575604f2',
 '8d43a757-e2c0-45fc-ab28-773de5599f6d',
 'fb17aefb-bcb2-4c7e-9a68-d3ad6ab8fc48',
 '5b0eba2a-3ad4-44b0-a616-1fc6645a17fd']

In [14]:
query_str = "who  are eligible for admission to Ph. D."

In [15]:
query_embedding = embed_model.get_query_embedding(query_str)

In [16]:

from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"


vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [17]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

Regulations for the award of the Degree of Doctor of Philosophy by the A P J
Abdul Kalam Technological University 2023.
(Revised as per the University Grants Commission (Minimum Standards and Procedures for Award
of Ph.D. Degree) Regulations, 2022)
1.
Admission
1.1
Categories of Admission
1.1.1
There shall be TWO categories of Ph. D. admission as given below:
(i)
Full-time scholars with or without fellowship
(ii) Part-time scholars in the following category admitted to pursue research towards Ph.
D. degree while working.
a.
Faculty members of the University Departments 
b.
Faculty members of affiliated colleges 
c.
Persons working with Organisations / Industries within the Country 
1.2 
Ph.D. through Part-time 
1.2.1 
Ph.D. programmes through part-time will be permitted, provided all the conditions stipulated
in the Regulations are fulfilled.
1.2.3 
The candidate shall produce a “No Objection Certificate” from the appointing authority in
the organization where the candidate is employed

In [18]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [19]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [20]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [21]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [22]:
query_str = "who  are eligible for admission to Ph. D."

response = query_engine.query(query_str)


llama_print_timings:        load time =  204988.22 ms
llama_print_timings:      sample time =      34.33 ms /    57 runs   (    0.60 ms per token,  1660.21 tokens per second)
llama_print_timings: prompt eval time =  766119.10 ms /  1941 tokens (  394.70 ms per token,     2.53 tokens per second)
llama_print_timings:        eval time =   15256.38 ms /    56 runs   (  272.44 ms per token,     3.67 tokens per second)
llama_print_timings:       total time =  781459.20 ms /  1997 tokens


In [23]:
print(str(response))

 Candidates with a Master’s degree in Engineering/Technology, Architecture or a Master’s degree by research in Engineering/Technology with a minimum CGPA of 5.75 are eligible for admission to Ph.D. programs.


In [24]:
print(response.source_nodes[0].get_content())

Regulations for the award of the Degree of Doctor of Philosophy by the A P J
Abdul Kalam Technological University 2023.
(Revised as per the University Grants Commission (Minimum Standards and Procedures for Award
of Ph.D. Degree) Regulations, 2022)
1.
Admission
1.1
Categories of Admission
1.1.1
There shall be TWO categories of Ph. D. admission as given below:
(i)
Full-time scholars with or without fellowship
(ii) Part-time scholars in the following category admitted to pursue research towards Ph.
D. degree while working.
a.
Faculty members of the University Departments 
b.
Faculty members of affiliated colleges 
c.
Persons working with Organisations / Industries within the Country 
1.2 
Ph.D. through Part-time 
1.2.1 
Ph.D. programmes through part-time will be permitted, provided all the conditions stipulated
in the Regulations are fulfilled.
1.2.3 
The candidate shall produce a “No Objection Certificate” from the appointing authority in
the organization where the candidate is employed